# Treasury Trading Optimization: Python Comparison
This notebook compares optimization engines (MOSEK, Gurobi, CPLEX, Xpress, SCIP, CVXOPT, KNITRO) for a convex optimization problem in treasury trading using 2024 U.S. Treasury real yield curve data. The objective is to minimize funding costs while accounting for haircuts, interest rates, and transfer fees.

In [ ]:
!pip install pandas numpy requests mosek gurobipy cplex fico-xpress pyscipopt cvxopt knitro matplotlib seaborn

## Data Retrieval

In [ ]:
import pandas as pd
import requests

# URL for 2024 Treasury real yield curve data
url = "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/2024/all?type=daily_treasury_real_yield_curve&field_tdr_date_value=2024&page&_format=csv"
response = requests.get(url)
with open("treasury_2024.csv", "wb") as f:
    f.write(response.content)

# Load and process data
data = pd.read_csv("treasury_2024.csv")
data["Date"] = pd.to_datetime(data["Date"])
rates = data[["Date", "5 Yr", "10 Yr", "20 Yr", "30 Yr"]].dropna()
print(rates.tail())

## Problem Definition
- **Objective**: Minimize funding costs = $\sum (w_i \cdot r_i + f_i)$
- **Constraints**:
  - $\sum w_i = 1$ (full allocation)
  - $w_i \geq h_i = 0.02$ (haircut constraint)
  - $w_i \geq 0$ (non-negative weights)
- **Parameters**:
  - $r_i$: Latest real yields (5, 10, 20, 30-year treasuries)
  - $h_i = 0.02$: Haircut of 2%
  - $f_i = 0.001$: Transfer fee of 0.1%

In [ ]:
import numpy as np
r = rates.iloc[-1, 1:].values / 100  # Latest rates in decimal
h = np.array([0.02] * 4)  # Haircuts
f = np.array([0.001] * 4)  # Fees
n = len(r)

## Optimization Engines

In [ ]:
# MOSEK
from mosek.fusion import Model, Domain, Expr
with Model("TreasuryOpt") as M:
    w = M.variable("w", n, Domain.greaterThan(0.0))
    M.constraint("budget", Expr.sum(w), Domain.equalsTo(1.0))
    M.constraint("haircut", w, Domain.greaterThan(h))
    cost = Expr.add(Expr.dot(w, r), Expr.constTerm(f.sum()))
    M.objective("obj", ObjectiveSense.Minimize, cost)
    M.solve()
    w_mosek = w.level()
    cost_mosek = M.primalObjValue()
print(f"MOSEK: Weights = {w_mosek}, Cost = {cost_mosek}")

In [ ]:
# Gurobi
from gurobipy import Model, GRB
m = Model("TreasuryOpt")
w = m.addVars(n, lb=0, name="w")
m.addConstr(sum(w[i] for i in range(n)) == 1, "budget")
for i in range(n):
    m.addConstr(w[i] >= h[i], f"haircut_{i}")
m.setObjective(sum(w[i] * r[i] for i in range(n)) + f.sum(), GRB.MINIMIZE)
m.optimize()
w_gurobi = [w[i].x for i in range(n)]
cost_gurobi = m.objVal
print(f"Gurobi: Weights = {w_gurobi}, Cost = {cost_gurobi}")

In [ ]:
# CPLEX
from cplex import Cplex
cp = Cplex()
w_idx = list(range(n))
cp.variables.add(names=[f"w_{i}" for i in w_idx], lb=[0]*n)
cp.linear_constraints.add(lin_expr=[[ ["w_" + str(i) for i in w_idx], [1]*n]], senses=["E"], rhs=[1])
for i in w_idx:
    cp.linear_constraints.add(lin_expr=[[ [f"w_{i}"], [1]]], senses=["G"], rhs=[h[i]])
cp.objective.set_sense(cp.objective.sense.minimize)
cp.objective.set_linear([(f"w_{i}", r[i]) for i in w_idx])
cp.objective.set_offset(f.sum())
cp.solve()
w_cplex = cp.solution.get_values()
cost_cplex = cp.solution.get_objective_value()
print(f"CPLEX: Weights = {w_cplex}, Cost = {cost_cplex}")

In [ ]:
# Xpress
import xpress as xp
p = xp.problem()
w = [xp.var(lb=0) for _ in range(n)]
p.addVariable(w)
p.addConstraint(xp.Sum(w) == 1)
for i in range(n):
    p.addConstraint(w[i] >= h[i])
p.setObjective(xp.Sum(w[i] * r[i] for i in range(n)) + f.sum(), sense=xp.minimize)
p.solve()
w_xpress = [p.getSolution(w[i]) for i in range(n)]
cost_xpress = p.getObjVal()
print(f"Xpress: Weights = {w_xpress}, Cost = {cost_xpress}")

In [ ]:
# SCIP
from pyscipopt import Model
m = Model("TreasuryOpt")
w = [m.addVar(f"w_{i}", lb=0) for i in range(n)]
m.addCons(sum(w) == 1, "budget")
for i in range(n):
    m.addCons(w[i] >= h[i], f"haircut_{i}")
m.setObjective(sum(w[i] * r[i] for i in range(n)) + f.sum(), "minimize")
m.optimize()
w_scip = [m.getVal(w[i]) for i in range(n)]
cost_scip = m.getObjVal()
print(f"SCIP: Weights = {w_scip}, Cost = {cost_scip}")

In [ ]:
# CVXOPT
from cvxopt import matrix, solvers
P = matrix(0.0, (n, n))  # No quadratic term
q = matrix(r + f)
G = matrix(np.vstack([-np.eye(n), np.ones((1, n))]))
h_cvx = matrix(np.hstack([-h, [1]]))
A = matrix(1.0, (1, n))
b = matrix(1.0)
sol = solvers.qp(P, q, G, h_cvx, A, b)
w_cvxopt = np.array(sol["x"]).flatten()
cost_cvxopt = (r @ w_cvxopt + f.sum())
print(f"CVXOPT: Weights = {w_cvxopt}, Cost = {cost_cvxopt}")

In [ ]:
# KNITRO
from knitro import *
kc = KN_new()
var = [KN_add_var(kc) for _ in range(n)]
KN_add_con(kc, lambda x: sum(x) - 1, [var])
for i in range(n):
    KN_add_con(kc, lambda x, i=i: x[i] - h[i], [[var[i]]])
KN_set_obj(kc, lambda x: sum(x[i] * r[i] for i in range(n)) + f.sum())
KN_set_var_lobnds(kc, var, [0]*n)
KN_solve(kc)
w_knitro = KN_get_solution(kc)[1]
cost_knitro = KN_get_obj_value(kc)
print(f"KNITRO: Weights = {w_knitro}, Cost = {cost_knitro}")
KN_free(kc)

## Results and Visualization

In [ ]:
results = {
    "MOSEK": (w_mosek, cost_mosek),
    "Gurobi": (w_gurobi, cost_gurobi),
    "CPLEX": (w_cplex, cost_cplex),
    "Xpress": (w_xpress, cost_xpress),
    "SCIP": (w_scip, cost_scip),
    "CVXOPT": (w_cvxopt, cost_cvxopt),
    "KNITRO": (w_knitro, cost_knitro)
}

# Cost comparison
df_cost = pd.DataFrame({k: [v[1]] for k, v in results.items()}, index=["Cost"]).T

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.barplot(x=df_cost.index, y="Cost", data=df_cost)
plt.title("Optimization Engine Cost Comparison (Feb 19, 2025)")
plt.ylabel("Total Funding Cost")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Weights comparison
weights_df = pd.DataFrame({k: v[0] for k, v in results.items()}, index=["5 Yr", "10 Yr", "20 Yr", "30 Yr"]).T
plt.figure(figsize=(12, 6))
sns.heatmap(weights_df, annot=True, cmap="YlGnBu", fmt=".3f")
plt.title("Allocation Weights by Engine")
plt.tight_layout()
plt.show()

## Notes
- **Data**: Latest rates from 2024 used. Adjust URL if format changes.
- **Licenses**: MOSEK, Gurobi, CPLEX, Xpress, KNITRO need licenses; SCIP and CVXOPT are free.
- **Performance**: All solvers should yield similar results for this linear problem.